<h1>Table of Contents<span class="tocSkip"></span></h1>


# Introduction
<hr style = "border:2px solid black" ></hr>


**What?** Scipy.optimize vs. CVXPY


# Imports
<hr style = "border:2px solid black" ></hr>

In [1]:
import numpy as np
from scipy.optimize import minimize
import time
import numpy as np
from cvxpy import *  

# Scipy.optimize
<hr style = "border:2px solid black" ></hr>

## No Jacobian

In [35]:
matrix = np.array([[1.0, 1.5, -2.],
                   [0.5, 3.0, 2.5],
                   [1.0, 0.25, 0.75]])


def fct(x):
    return x.dot(matrix).dot(x)


x0 = np.ones(3) / 3
print(x0)
cons = ({'type': 'eq', 'fun': lambda x: x.sum() - 1.0})
bnds = [(0, 1)] * 3

w = minimize(fct, x0, method='SLSQP', bounds=bnds, constraints=cons)['x']

[0.33333333 0.33333333 0.33333333]


In [8]:
w

array([4.54545454e-01, 1.64104841e-15, 5.45454546e-01])

## With Jacobian of only function + (constraint will be estimated via FDs)

In [36]:
def fct_deriv(x):
    return 2 * matrix.dot(x)

In [37]:
minimize(fct, x0, method='SLSQP', jac=fct_deriv,
             bounds=bnds, constraints=cons)['x']

array([0.65517408, 0.17241322, 0.17241269])

## With Jacobian of both function + constraints

In [38]:
cons2 = ({'type': 'eq', 'fun': lambda x: x.sum() -
         1.0, 'jac': lambda x: np.ones_like(x)})

In [39]:
minimize(fct, x0, method='SLSQP', jac=fct_deriv,
             bounds=bnds, constraints=cons2)['x']

array([0.65517408, 0.17241322, 0.17241269])

# CVXPY
<hr style = "border:2px solid black" ></hr>

In [4]:
# Convex-Opt

""" Create some random pos-def matrix """
N = 3
matrix_ = np.random.normal(size=(N,N))
matrix = np.dot(matrix_, matrix_.T)

""" CVXPY-based Convex-Opt """
x = Variable(N)
constraints = [x >= 0, x <= 1, sum(x) == 1]
objective = Minimize(quad_form(x, matrix))
problem = Problem(objective, constraints)
time_start = time.perf_counter()
problem.solve(solver=SCS, use_indirect=True, verbose=True)  # or: solver=ECOS
time_end = time.perf_counter()
print(problem.value)
print('cvxpy (modelling) + ecos/scs (solving) used (secs): ', time_end - time_start)



cvxpy

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Sep 08 08:56:50 PM: Your problem has 3 variables, 3 constraints, and 0 parameters.
(CVXPY) Sep 08 08:56:50 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 08 08:56:50 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 08 08:56:50 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 08 08:56:50 PM: Compiling problem (target solver=SCS).
(CVXPY) Sep 08 08:56:50 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuff

In [27]:
problem.value

0.01524205629353699

# References
<hr style = "border:2px solid black" ></hr>


- https://stackoverflow.com/questions/43648073/what-is-the-fastest-way-to-minimize-a-function-in-python

